In [1]:
import csv
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
age_train = pd.read_csv('./csv_data/age_train.csv',header=None)
age_train.columns = ['uId','age_group']
app_info = pd.read_csv('./csv_data/app_info.csv',header=None)
app_info.columns = ['appId','category']
user_basic_info = pd.read_csv('./csv_data/user_basic_info.csv',header=None)
user_basic_info.columns = ['uId','gender','city','prodName',
                           'ramCapacity','ramLeftRation','romCapacity',
                           'romLeftRation','color','fontSize',
                           'ct','carrier','os']
user_app_actived = pd.read_csv('./csv_data/user_app_actived.csv',header=None)
user_app_actived.columns = ['uId','appId']
user_behavior_info = pd.read_csv('./csv_data/user_behavior_info.csv',header=None)
user_behavior_info.columns = ['uId','bootTimes','AFuncTimes','BFuncTimes',
                              'CFuncTimes','DFuncTimes','EFuncTimes',
                              'FFuncTimes','FFuncSum']

In [3]:
# user_basic_info = user_basic_info.dropna()
# app_info = app_info.dropna()
# age_train = age_train.dropna()
# user_behavior_info = user_behavior_info.dropna()
# user_app_actived = user_app_actived.dropna()

app_info = pd.get_dummies(app_info,columns=['category'])
del user_basic_info['city']
user_basic_info = pd.get_dummies(user_basic_info,columns=['prodName','color','ct','carrier'])
# user_basic_info = pd.get_dummies(user_basic_info,columns=['city','prodName','color','ct','carrier'])
# todo: city 和 prodName 维度太高，估计比较稀疏，要把数目少的类别归为一个other类。
# city这个特征作用不大，而且有300多维，可能可以删了

# df = user_behavior_info
# float_col = [each for each in df.columns if df[each].dtype=='float']
# normalize_col = float_col + ['bootTimes','FFuncSum']
# user_behavior_info[normalize_col] = df[normalize_col].apply(lambda x: (x - np.nanmean(x)) / np.nanstd(x))

# df = user_basic_info
# float_col = [each for each in df.columns if df[each].dtype=='float']
# user_basic_info[float_col] = df[float_col].apply(lambda x: (x - np.nanmean(x)) / np.nanstd(x))

In [4]:
app_info_dict = {}
for each in app_info.values:
    app = each[0]
    cat = each[1:]
    if app in app_info.keys():
        app_info_dict[app] += cat
    else:
        app_info_dict[app] = cat
        
    
def f(x):
    x = x.split('#')
    r = []
    for each in x:
        if each in app_info_dict.keys():
            r.append(app_info_dict[each])
        else:
            r.append(np.array([0]*40))
    return sum(r)
    
user_app_actived['app_total_categery'] = user_app_actived['appId'].apply(f)
user_app_actived['app_numbers'] = user_app_actived['app_total_categery'].apply(lambda x:sum(x))
user_app_actived['app_average_categery'] = user_app_actived['app_total_categery'].apply(lambda x:x/np.nansum(x))

In [10]:
top100List = ['a00289791', 'a00103666', 'a00289826', 'a00287085', 'a00278905', 'a0048562', 'a00289850', 'a0046242', 'a00170432', 'a00278858', 'a00290015', 'a0030644', 'a00247519', 'a00271087', 'a0048332', 'a00109386', 'a00289866', 'a00292780', 'a00224427', 'a00290027', 'a00187480', 'a00261220', 'a0021880', 'a00274701', 'a00348507', 'a00292472', 'a00336224', 'a00289511', 'a0022276', 'a0071100', 'a00311680', 'a00481059', 'a0036392', 'a00275200', 'a00278849', 'a00157201', 'a00244790', 'a00263057', 'a00289519', 'a00331160', 'a00287057', 'a0048522', 'a009932', 'a00307756', 'a0048467', 'a00217423', 'a00363352', 'a00166554', 'a00299200', 'a0027551', 'a003925', 'a0092898', 'a0064339', 'a00144187', 'a0048637', 'a00461814', 'a00271777', 'a00373197', 'a00157220', 'a00289728', 'a00141402', 'a0071095', 'a00276196', 'a00290038', 'a0082383', 'a00324307', 'a00299316', 'a0099559', 'a00336168', 'a00281804', 'a0032847', 'a00169965', 'a0075197', 'a00294247', 'a00331594', 'a00278867', 'a00278912', 'a00160089', 'a00261226', 'a00219676', 'a0049622', 'a0082007', 'a00239261', 'a0048452', 'a00348136', 'a00248909', 'a00287071', 'a00135785', 'a00235015', 'a00287090', 'a00101827', 'a00289466', 'a00225310', 'a00153189', 'a0029508', 'a0017114', 'a00294240', 'a00158371', 'a00290067', 'a00225081']

In [21]:
len(top100List)

100

In [19]:
def f1(x):
    x = x.split('#')
    r = [top100List.index(each) for each in set(x).intersection(top100List)]
    a = [0]*100
    for each in r:
        a[each] = 1
    return np.array(a)
user_app_actived['top100_app'] = user_app_actived['appId'].apply(f1)

In [22]:
temp = user_app_actived[['uId','app_average_categery']].values
temp1 = []
for i in  range(len(temp)):
    temp1.append(np.insert(temp[i][1],0,temp[i][0]))
temp1 = np.array(temp1)
temp1 = pd.DataFrame(temp1)
temp1.columns = ['uId'] + ['categery_'+str(i) for i in range(40)]
temp1['uId'] = temp1['uId'].astype('int64')
temp1[['categery_'+str(i) for i in range(40)]] = temp1[['categery_'+str(i) for i in range(40)]].astype('float64')
 
user_app_actived = pd.merge(user_app_actived,temp1, on=['uId'], how='inner')

temp = user_app_actived[['uId','top100_app']].values
temp1 = []
for i in  range(len(temp)):
    temp1.append(np.insert(temp[i][1],0,temp[i][0]))
temp1 = np.array(temp1)
temp1 = pd.DataFrame(temp1)
temp1.columns = ['uId'] + ['top100_app_'+str(i) for i in range(100)]
temp1['uId'] = temp1['uId'].astype('int64')
temp1[['top100_app_'+str(i) for i in range(100)]] = temp1[['top100_app_'+str(i) for i in range(100)]].astype('float64')
 
user_app_actived = pd.merge(user_app_actived,temp1, on=['uId'], how='inner')
del user_app_actived['app_total_categery']
del user_app_actived['app_average_categery']
del user_app_actived['appId']
del user_app_actived['top100_app']

In [30]:
user_app_actived.head()

,uId,app_numbers,categery_0,categery_1,categery_2,categery_3,categery_4,categery_5,categery_6,categery_7,...,top100_app_90,top100_app_91,top100_app_92,top100_app_93,top100_app_94,top100_app_95,top100_app_96,top100_app_97,top100_app_98,top100_app_99
0,1000008,45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1000010,44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1000011,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000013,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024390,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1000014,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
user_app_actived.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2512500 entries, 0 to 2512499
Columns: 142 entries, uId to top100_app_99
dtypes: float32(140), uint16(1), uint32(1)
memory usage: 1.3 GB


In [29]:
user_app_actived = optimize_mem(user_app_actived)
with open('user_app_actived.pkl','wb') as f:
    pickle.dump(user_app_actived,f)

2741.15 MB
1375.37 MB


In [15]:
user_app_usage_file = ['./user_app_usage_features_na/'+each for each in os.listdir('./user_app_usage_features_na/') if each.endswith('.pkl')]
# user_app_usage_file.remove('./user_app_usage_features_na/dataset.pkl')
# user_app_usage_file.remove('./user_app_usage_features_na/user_app_usage_features.pkl')
user_app_usage_file.sort()
print(user_app_usage_file)
temp = []
for each in user_app_usage_file:
    with open(each,'rb') as f:
        temp.append(pickle.load(f))
user_app_usage = pd.concat(temp)
user_app_usage = user_app_usage[list(user_app_usage.columns)].astype('float64')
# user_app_usage = user_app_usage.dropna()

['./user_app_usage_features_na/user_app_usage_features0.pkl', './user_app_usage_features_na/user_app_usage_features1.pkl', './user_app_usage_features_na/user_app_usage_features10.pkl', './user_app_usage_features_na/user_app_usage_features100.pkl', './user_app_usage_features_na/user_app_usage_features101.pkl', './user_app_usage_features_na/user_app_usage_features102.pkl', './user_app_usage_features_na/user_app_usage_features103.pkl', './user_app_usage_features_na/user_app_usage_features104.pkl', './user_app_usage_features_na/user_app_usage_features105.pkl', './user_app_usage_features_na/user_app_usage_features106.pkl', './user_app_usage_features_na/user_app_usage_features107.pkl', './user_app_usage_features_na/user_app_usage_features108.pkl', './user_app_usage_features_na/user_app_usage_features109.pkl', './user_app_usage_features_na/user_app_usage_features11.pkl', './user_app_usage_features_na/user_app_usage_features110.pkl', './user_app_usage_features_na/user_app_usage_features111.pkl

In [22]:
# df = user_app_usage
# float_col = [each for each in df.columns if df[each].dtype=='float']
# user_app_usage[float_col] = df[float_col].apply(lambda x: (x - np.nanmean(x)) / np.nanstd(x))
user_app_usage = user_app_usage.reset_index()
user_app_usage = user_app_usage.rename(columns={'':'uId'})

In [23]:
user_app_usage.head()

,uId,weekdayDuration_0,weekdayDuration_1,weekdayDuration_2,weekdayDuration_3,weekdayDuration_4,weekdayDuration_5,weekdayDuration_6,weekdayDuration_7,weekdayDuration_8,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
0,1000237,-0.319478,-0.225876,-0.332654,2.160660,-0.069964,-0.124293,-0.219332,-0.275030,-0.231208,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079802,-0.089865,0.062155,0.210613,-0.030982
1,1000242,-0.992592,-0.431682,-0.362771,-0.421549,-0.433786,-0.268710,-0.219332,-0.275030,-0.293687,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079802,-0.089865,-1.079389,-1.108099,1.658166
2,1000754,-0.706378,0.016601,-0.362771,2.297029,1.288722,-0.226560,-0.219332,-0.067255,-0.293687,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079802,-0.089865,0.321683,0.428305,-0.026958
3,1001019,1.217951,0.143937,-0.195281,-0.243414,-0.115189,-0.268710,-0.219332,-0.214891,0.004599,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079802,-0.089865,1.396545,1.567263,-0.027400
4,1001253,0.853085,0.150076,-0.358390,0.240420,0.382198,-0.249979,0.541790,1.536093,-0.293687,...,-0.118402,-0.04991,4.218220,-0.087259,-0.08037,-0.079802,2.398098,0.619287,0.325215,0.001310


In [21]:
user_app_usage.head()

,weekdayDuration_0,weekdayDuration_1,weekdayDuration_2,weekdayDuration_3,weekdayDuration_4,weekdayDuration_5,weekdayDuration_6,weekdayDuration_7,weekdayDuration_8,weekdayDuration_9,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
,,,,,,,,,,,,,,,,,,,,,
1000237,58033.0,2447.0,1258.0,61578.0,12067.0,5297.0,0.0,0.0,465.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12772.0,5055.0,2.526607
1000242,3227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720.0,15.0,48.000000
1000754,26531.0,5330.0,0.0,64830.0,57131.0,1546.0,0.0,2909.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15512.0,5887.0,2.634958
1001019,183213.0,6844.0,6996.0,4248.0,10567.0,0.0,0.0,842.0,2220.0,49653.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26860.0,10240.0,2.623047
1001253,153505.0,6917.0,183.0,15786.0,27064.0,687.0,23626.0,25357.0,0.0,0.0,...,0.0,0.0,96.0,0.0,0.0,0.0,36.0,18654.0,5493.0,3.395958


In [71]:
user_basic_info.head()

,uId,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,os,prodName_p001,prodName_p0010,prodName_p00100,...,carrier_China_Mobile,carrier_China_Telecom,carrier_China_Unicom,carrier_othercp,fontSize_0.0,fontSize_0.85,fontSize_1.0,fontSize_1.15,fontSize_1.3,fontSize_1.30001
0,1000001,0,3.0,0.43,32.0,0.46,8.0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1000002,1,NaN,0.26,NaN,0.59,9.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,1000004,1,3.0,0.36,32.0,0.09,8.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,1000006,0,2.0,0.19,16.0,0.42,6.0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,1000007,1,6.0,0.20,137.0,0.50,9.0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [65]:
feature = user_basic_info['fontSize'].value_counts()
feature_select = list(feature[feature>=5000].index)
feature_select.append(np.nan)
user_basic_info['fontSize'][-(user_basic_info['fontSize'].isin(feature_select))] = 0
user_basic_info = pd.get_dummies(user_basic_info,columns=['fontSize'])

/home/haha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [79]:
user_basic_info['os'].value_counts()

8.0     1115342
9.0      767572
8.1      201916
6.0      184700
7.0      172461
4.4       36217
5.1       25295
5.0        5296
4.2        1869
4.3         430
4.1          77
7.1          72
4.0          16
6.3           4
6.1           4
12.1          1
Name: os, dtype: int64

In [67]:
feature

1.00000    1289211
0.00000     615679
1.15000     276456
1.30000     142361
1.30001     135958
0.85000      52835
Name: fontSize, dtype: int64

1. user_behavior_info,  
2. user_basic_info,  
3. user_app_actived,  
4. user_app_usage

In [51]:
join_data = age_train
print(len(join_data))
join_data = pd.merge(join_data,user_behavior_info, on=['uId'], how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_basic_info,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_actived,on=['uId'],how='inner')
print(len(join_data))
join_data = pd.merge(join_data,user_app_usage,on=['uId'],how='inner')
print(len(join_data))
# join_data.dropna()
print(len(join_data))

train_data = join_data
train_data = optimize_mem(train_data)

2010000
2010000
2010000
2010000
1582576
1582576
6178.92 MB
3365.65 MB


In [16]:
with open('train.data.na','wb') as f:
    pickle.dump(train_data,f)

NameError: name 'train_data' is not defined

In [17]:
age_test = pd.read_csv('./csv_data/age_test.csv',header=None)
age_test.columns = ['uId']

In [31]:
len(user_app_usage)

1977442

In [32]:
len(set(test_data['uId']))

502500

In [24]:
join_data = age_test
print(len(join_data))
join_data = pd.merge(join_data,user_behavior_info, on=['uId'], how='left')
print(len(join_data))
join_data = pd.merge(join_data,user_basic_info,on=['uId'],how='left')
print(len(join_data))
join_data = pd.merge(join_data,user_app_actived,on=['uId'],how='left')
print(len(join_data))
join_data = pd.merge(join_data,user_app_usage,on=['uId'],how='left')
print(len(join_data))
# join_data.dropna()
# print(len(join_data))

test_data = join_data
test_data = optimize_mem(test_data)

502500
502500
502500
502500
502526
1958.20 MB
1068.24 MB


In [33]:
test_data.tail(10)

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,gender,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
502516,3180139,1.165865,-0.524734,0.093684,0.187687,-0.230666,-0.229489,-0.167348,0.225306,1,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,0.494259,0.229190,0.000776
502517,3180191,-0.085970,0.300054,0.005132,-0.080571,-0.210429,-0.148539,-0.167348,-0.107233,0,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,0.138971,0.585817,-0.047080
502518,3180290,0.378997,-0.524734,-0.021434,-0.183851,-0.096093,-0.162955,0.547600,-0.027520,0,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,0.984521,0.695448,-0.003794
502519,3180573,-0.550937,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,1,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,-1.145407,-1.106006,-0.087690
502520,3180722,-0.264804,-0.524734,-0.109986,-0.161049,-0.210429,-0.229489,-0.167348,-0.103986,1,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,-1.052773,-0.711962,-0.100517
502521,3180767,-0.229037,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502522,3181007,-0.729771,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.128223,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502523,3181086,0.307464,-0.524734,-0.083420,-0.187875,0.022290,0.099856,-0.167348,-0.119436,0,...,-0.118402,-0.04991,-0.097799,-0.087259,-0.08037,-0.079801,-0.089865,-0.653915,-0.799353,0.037176
502524,3181099,-0.515171,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134021,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502525,3181144,-0.729771,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
test_data.head()

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,gender,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
0,1000002,-0.229037,-0.524734,0.040552,0.446556,-0.106211,0.202984,0.547600,-0.131471,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000003,-0.264804,0.300054,-0.109986,0.822118,0.072881,-0.040975,-0.167348,-0.066949,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000004,1.308932,-0.524734,-0.083420,-0.030944,-0.166921,-0.229489,-0.167348,-0.050398,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000005,-0.729771,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000006,0.450531,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
with open('test.data.na','wb') as f:
    pickle.dump(test_data,f)

In [36]:
test_data.head()

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,gender,...,weekendTimes_93,weekendTimes_94,weekendTimes_95,weekendTimes_96,weekendTimes_97,weekendTimes_98,weekendTimes_99,weekdayTimesSum,weekendTimesSum,rate2
0,1000002,-0.229037,-0.524734,0.040552,0.446556,-0.106211,0.202984,0.547600,-0.131471,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000003,-0.264804,0.300054,-0.109986,0.822118,0.072881,-0.040975,-0.167348,-0.066949,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000004,1.308932,-0.524734,-0.083420,-0.030944,-0.166921,-0.229489,-0.167348,-0.050398,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000005,-0.729771,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000006,0.450531,-0.524734,-0.109986,-0.214701,-0.230666,-0.229489,-0.167348,-0.134224,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # 我们假设这不是一个df，而是一个 Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # 将 bytes 转化成 megabytes
    return "{:03.2f} MB".format(usage_mb)

def optimize_mem(sll):
    print(mem_usage(sll))
    sll_int = sll.select_dtypes(include=['int']) # 用 DataFrame.select_dtypes 来选中表中的 int数据
    converted_int = sll_int.apply(pd.to_numeric,downcast='unsigned') # 用pd.to_numeric()来降低我们的数据类型

    compare_ints = pd.concat([sll_int.dtypes,converted_int.dtypes],axis=1)
    compare_ints.columns = ['before','after']
    
    sll_float = sll.select_dtypes(include=['float'])
    converted_float = sll_float.apply(pd.to_numeric,downcast='float')

    compare_floats = pd.concat([sll_float.dtypes,converted_float.dtypes],axis=1)
    compare_floats.columns = ['before','after']
    
    sll[converted_int.columns] = converted_int
    sll[converted_float.columns] = converted_float

    print(mem_usage(sll))
    return sll